In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade plotly 

In [ ]:
# Let's import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
from pandas_profiling import ProfileReport

In [ ]:
# Style des graphiques seaborn
sns.set_theme(style = "whitegrid")

In [ ]:
data_stroke = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
data_frame = data_stroke.copy()

# Démarche de travail :
## Objectif mesurable :
- **Objectif** : Prédire si une personne est atteinte d'un accident vasculaire cérébral ou pas à partir des données personnelles et cliniques fournies (problème de classification). Il est essentiel de noter qu'il est plus urgent, dans ce cas de figure, de détecter toutes les personnes malades (et non ceux qui sont vraiment malades parmi ceux qui sont identifiés comme malade). Sinon un patient malade peut être diagnostiquer comme sain alors que ce n'est pas le cas. Nous préférerons ainsi la sensibilité à la précision. 
- **Métrique et score à atteindre** : La proportion de la classe positive est largement inférieure à la proportion de la classe négative (nous notons une forte déséquilibre de classes). Dans ce cas de figure, la métrique accuracy ne sera pas assez performante pour évaluer notre modèle de classification. A la place, nous allons utiliser les métriques sensibilité et précision pour valider notre modèle :
    - Sensibilité : $\frac{VP}{VP+FN}$. Permet de calculer le pourcentage de tests positifs parmi les patients réellement atteints d'AVC ;
    - Précision : $\frac{VP}{VP+FP}$. Permet de calculer le pourcentage de patients réellement malades parmi ceux dont le test est positif ;
    - F1-score : $\frac{2\times Sensibilite \times Precision}{Sensibilite+Precision}$ ;
    - Avec :
        - $VP$ : Nombre de Vrais Positifs ;
        - $FN$ : Nombre de Faux Négatifs ;
        - $FP$ : Nombre de Faux Positifs ;
    
  Nous nous fixons une sensibilité supérieure à **80%**.
## Exploration des données :
### Analyse de la forme :

- Cible : La variable dépendante est la variable **stroke** qui contient des données discrètes. Cette variable indique si une personne est atteinte d'un accident vasculaire cérébral (1 pour atteinte) ou pas (0 pour non atteinte). Nous allons abréger, par la suite, accident vasculaire cérébral par AVC pour faciliter l'écrit. Nous devons transformer, plus tard, la variable cible en variable catégorielle non ordinale (cela nous permettra de différencier la catégorie "être atteint(e) d'AVC" de la catégorie "être non atteint(e) d'AVC"). Nous devrons ainsi utiliser un modèle de classification (classification model) pour déterminer si un patient est malade ou sain.

- Nombre de lignes et de colonnes : Nous avons identifié 5110 observations et 12 variables (dont la cible). Le nombre d'observations est inférieur à 10000, donc notre dataset ne contient pas beaucoup d'observations mais assez pour entraîner un modèle.
- Types de variables (à l'exception de la variable stroke) : Parmi les features, nous avons identifié 7 variables catégorielles dont deux contiennent des valeurs discrètes et les autres contiennent des valeurs de type object (chaîne de caractères) et 3 variables non catégorielles. 

    - Les variables de type object ont pour valeurs possibles les suivantes :
    
        - **gender** (genre) : contient les valeurs *Male* (Homme), *Female* (Femelle) ou *Other* (ni Homme, ni Femme) ;
    
        - **ever_married** (jamais marié(e)) : peut prendre les valeurs *Yes* (Oui) ou *No* (Non) ;
    
        - **work_type** (type de travail) : peut prendre les valeurs *Private* (Privée), *Self_employed* (Auto emploi ou Travail autonome), *Govt_job* (Travail au gouvernement), **children** (enfant), **Never_worked** (N'a jamais travaillé(e)) ;
    
        - **Residence_type** (type de résidence) : peut prendre les valeurs *Urban* (Urbaine), *Rural* (Rurale) ;
    
        - **smoking_status** (statut de fumeur) : contient les valeurs *formerly_smoked* (a fumé(e) dans le passé), *never_smoke* (n'a jamais fumé), *smokes* (fume), *Unknow* (donnée non recueillie).
    
    - Quant aux variables catégorielles **hypertension** et **heart_disease** (maladie de coeur), elles peuvent prendre les valeurs 1 (pour *positive*) ou 0 (pour *négative*). 
    
    - Nous identifions une variable discrète non categorielle (**age**) dont les valeurs varient de 0.08 (enfant de 9 mois) à 82 (adulte de 82 ans). Sa valeur moyenne est de 43 (adulte de 43 ans).
    
    - Le dataset ne contient que deux variables continues, **avg_glucose_level** (niveau moyen de glucose dans le sang) et **bmi** (indice de masse corporelle). Ces deux variables ne s'expriment pas avec les mêmes unités :
    
      - La variable avg_glucose_level a pour valeur maximale 271.74 et pour valeur minimale 55.12. Sa valeur moyenne est de 106.15.
      
      - La variable bmi a pour valeur maximale 97.60 et pour valeur minimale 10.30. Sa valeur moyenne est 28.89.
     
    - Nous définirons plus en détail ces variables dans la partie analyse du fond.
    
- Identification des valeurs manquantes : Nous remarquons que seule la variable bmi contient des valeurs manquantes qui sont un peu dispersées dans le dataset (Un peu entassées au niveau des premières observations). Mais en sachant que le nombre de valeurs manquantes ne représente que 16% dans l'ensemble des données présentes dans la colonne bmi et que nous allons choisir un échantillon aléatoire pour l'entraînement du modèle (ainsi que pour l'évaluation) donc on peut supposer que le remplacement des valeurs manquantes par la valeur la plus fréquente dans la variable bmi constitue une bonne stratégie.

- Identification des observations redondantes : Les données ne contiennent pas d'observations dupliquées.
### Analyse du fond :

- Visualisation de la cible : Nous remarquons 95.13% de personnes non atteintes d'AVC contre seulement 4.87% de personnes atteintes d'AVC. La proportion de patients non malades est largement supérieure au nombre de patients non malades.

- Compréhension des différentes variables : 

    - id : La variable id contient des valeurs discrètes et identifie chaque observation de manière unique. Elle n'apporte aucune information supplémentaire et donc n'influence pas le fait qu'une personne soit malade ou non. La variable id doit être supprimée.
    
    - Variables catégorielles :
    
        - Hypertension : Cette variable indique si oui ou non, le patient souffre d'hypertension. Un patient est testé positif à l'hypertension si on constate à deux reprises, et pas dans le même jour, une tension systolique supérieure ou égale à 140 mm Hg et/ou une tension diastolique supérieure ou égale à 90 mm Hg. Selon les tests cliniques, une hypertension peut souvent être la cause d'AVC. Nous verrons par la suite si les données recueillies de cette variable sont fiables. La colonne hypertension contient 90% de tests négatifs et 10% de tests positifs.
        
        - heart_disease : La variable heart_disease indique si le patient est atteint de cardiopathie (maladie cardiaque) ou pas. Il y a différents types de cardiopathies et nous verrons si ces derniers peuvent influencer le risque d'attraper un AVC. La colonne heart_disease contient 95% de tests positifs contre seulement 5% de tests négatifs. Cependant, nous savons que l'hypertension non traitée peut causer la cardiopathie. Donc on peut supposer, d'hors et déja, que ces deux variables sont fortement corrélées (hypothèse à vérifier).
        
        - gender : La variable gender indique à quel sexe appartient le patient. On doit vérifier si le sexe du patient peut influencer le risque qu'il soit atteint d'AVC. La colonne gender est composée de 58% de femmes, de 41% d'hommes et très peu (presque 0%) de sexe de type autres.
        
        - ever_married : Cette variable indique si le patient a déja été marié. L'analyse de cette variable doit nous permettre de dire si le patient a plus de chance d'attraper un AVC en s'étant déja marié (dans le temps présent ou passé) ou pas. Nous notons 66% des patients qui se déclarent ne s'être jamais mariés et 34% qui se sont déja mariés. 
        
        - work_type : Elle indique le type de travail effectué par un patient. Nous avons identifié 57% de patients travaillant dans le secteur privé (professions et secteurs d'activité ne dépendant pas de l'Etat), 16% des patients effectuant du travail autonome (ils sont leurs propres employés), 13% des patients sont des enfants (on n'a pas plus d'informations sur la catégorie children mais on suppose pour l'instant que le patient est un enfant et donc qu'il n'a pas besoin de travailler), presque 13% des patients travaillent pour le gouvernement (dans le secteur public) et seulement 0.4% n'ont jamais travaillé. Pour les patients dont le type de travail est children, nous devons vérifier si leurs ages indiquent que ce sont des enfants ou pas (c'est à dire certains sont des adultes).   
        
        - residence_type : Cette variable indique le type de résidence du patient. 51% des patients résident dans un milieu urbain (ville) et 49% des patients résident dans un milieu rural (campagne). Les proportions de ces deux classes sont presque similaires.
        
        - smoking_type : Elle indique si le patient est/était un fumeur ou pas. 37% des patients n'ont jamais fumés, 30% des patients n'indique pas s'il fument, 17% des patients ont fumé auparavant et 15% des patients disent fumer au moment où on les interrogeait. La catégorie unknow (inconnue) peut constituer un problème car elle n'apporte aucune information utile. Nous vérifierons par la suite si cette variable apporte de l'information à notre modèle.
        
    - variables non catégorielles :
        
        - age : Nous constatons que les patients qui  sont agées entre 37 et 63 ans sont plus nombreux dans la base de données, suivis des patients qui sont agés entre 78 et 82 ans tandis que les plus jeunes sont les moins nombreux.  
        
        - avg_glucose_level : Le niveau moyen de glucose dans le sang indique si, oui ou non, le patient est atteint de diabète. Cette variable s'exprime en mg/dL (milligrammes par déciLitre). Les patients qui souffre de diabète ont un niveau moyen de glucose supérieur ou égale à 200 mg/dL. Par contre, un niveau moyen de glucose inférieur à 140 mg/dL est considéré comme normal. Nous constatons que la plupart des patients n'ont pas de diabète car la plus grande partie des patients ont un niveau moyen de glucose tournant autour de 75-87 mg/dL de sang. Nous remarquons qu'environ 10 à 20% des patients ont un niveau moyen de glucose tournant autour de 210-225 mg/dL. On peut considérer que ces derniers sont atteints de diabète.
        
        - bmi : L'indice de masse corporelle (IMC) permet d'indiquer la corpulence d'un patient. Il s'exprime en kg/m2 (kilogrammes par mètre carré). Une personne est considérée comme obèse si son IMC dépasse 30 kg/m2. La plupart des patients éxaminés ont un IMC situé autour de 29 kg/m2. Ces derniers présentent un cas de surpoids ou d'obésité modérée (dont les IMC sont respectivement situés entre 25 et 30 kg/m2 et entre 30 et 35 kg/m2).
        
- Etude plus poussée des variables avec pandas-profiling : ##
            
- Visualisation des relations variables_explicatives - variable_a_expliquer :
    
    - Relations Cible - variables catégorielles : 
        
        - Pour la variable work_type : Nous remarquons que les patients qui ont pour type de travail children ont plus de chance de ne pas attraper d'AVC que les patients qui effectuent d'autres types de travail. On a peu de patients non travailleurs dans la variable work_type donc on ne peut rien dire par rapport à cette catégorie (mais il est possible qu'elle influence aussi le fait qu'on soit atteint d'AVC ou pas). 
        
        - Pour la variable smoking_type : Nous remarquons que les patients ne disant pas s'ils fument/fumaient présentent une proportion de malades inconsistante par rapport aux autres types de fumeurs. Cela est dû au fait que la catégorie unknow n'est pas fiable (elle se comporte comme une valeur manquante).
        
    - Relations Cible - variables non catégorielles :
        
        - Parmi les variables non catégorielles, seule la variable age influence le fait qu'un patient soit atteint d'AVC ou pas. Il y a plus de risque d'attraper un AVC chez les patients plus âgés ;
        
        - Pour les deux autres variables restantes nous constatons que les distributions sont presque les mêmes pour les patients malades et non malades ;
        
        - Nous vérifierons plus amplement ces hypothèses à travers un test de student.
        
- Visualisation des relations entre variables quantitatives (corrélations) : Les variables quantitatives ne partagent aucune forte corrélation entre elles.

- Visualisation des relations entre variables catégorielles et quantitatives : Nous constatons que quelques variables catégorielles ont de fortes corrélations avec des variables quantitatives.

    - La variable **age** est fortement corrélée avec les variables **work_type**, **smoking_status** et **ever_married**. Pour sa relation avec les autres variables catégorielles nous remarquons de légères corrélations ou quasiment pas de corrélations pour certaines.
    
    - La variable **bmi** est, pour son cas, fortement corrélée avec les variables **ever_married** et **work_type**. 
    
    - En revenant au niveau de pandas profiling, nous constatons que nos analyses sont soutenues par les remarques faites par la librairie (aucune incohérence n'est à noter).
   
- Identification des valeurs aberrantes :

# Exploration des données

## Identification de la cible

In [ ]:
# Vérifions le contenu des dix premières lignes  
data_frame.head(10)

Nous savons que la cible est la variable stroke. C'est la variable à expliquer.

In [ ]:
# Déterminons le type de la cible et son contenu
# Type de la variable
data_frame["stroke"].dtype

In [ ]:
# Vérifions les valeurs possibles du target
data_frame["stroke"].unique()

## Nombre de ligne et de colonnes

In [ ]:
# Forme du dataframe
data_frame.shape

Nous avons 5110 observations et 12 variables (dont la cible)

## Types des variables explicatives

In [ ]:
# Identifions les types des variables
data_frame.dtypes

Nous identifions 7 colonnes de type catégorielles. Nous allons stocker les noms de ces colonnes dans une variable.

In [ ]:
categorical_columns = ["hypertension", "heart_disease"]
categorical_columns.extend(data_frame.select_dtypes('object').columns)

In [ ]:
# Pour chaque variable de type object vérifions ses catégories
for column in data_frame.select_dtypes('object').columns:
    print(f"Valeurs uniques de {column:-<20} {data_frame[column].unique()}\n")

In [ ]:
# Vérifions les valeurs que contient les variables hypertension et heart_disease
for column in categorical_columns[:2]:
    print(f"Unique values of {column:-<20} {data_frame[column].unique()}\n")

Stockons dans une autre variable les noms des colonnes non catégorielles

In [ ]:
non_categorical_columns = ["age", "avg_glucose_level", "bmi"]

In [ ]:
# Vérifions les statistiques des colonnes non catégorielles
data_frame[non_categorical_columns].describe()

## Identification des valeurs manquantes

Première analyse des valeurs manquantes par sommation

In [ ]:
data_frame.isna().sum()

In [ ]:
# Stockage du dataframe des données manquantes dans une variable
na_values = data_frame.isna()

Deuxième analyse par visualisation (Visualisation de la répartition des valeurs manquantes dans le dataset)

In [ ]:
px.imshow(na_values)

Nous remarquons que seule la variable bmi contient des valeurs manquantes qui sont un peu dispersées dans le dataset (Un peu entassées au niveau des premières observations). 

Vérifions le pourcentage de valeurs manquantes

In [ ]:
percentage_of_na = na_values["bmi"].sum()/data_frame.shape[1]

In [ ]:
print(f"La variable bmi contient {percentage_of_na}% de valeurs manquantes.")

## Identification des observations redondantes

In [ ]:
# Vérifions s'il y a des observations dupliquées dans le dataset
data_frame.duplicated().sum()

Le dataset ne contient pas d'observations dupliquées.

## Visualisation de la cible

Nous devons voir la proportion de chaque classe au sein de la variable cible pour identifier si on a affaire à un désiquilibre de classe.

In [ ]:
# Vérifions la proportion de chaque classe
proportions = data_frame["stroke"].value_counts(normalize = True)*100
proportions

Visualisons la proportion de chaque classe à l'aide d'un diagramme en camembert

In [ ]:
proportions.plot.pie()

Nous remarquons que 95.13% des personnes sont atteintes d'AVC contre seulement 4.87% de personnes atteintes d'AVC.

In [ ]:
# Transformation en variable catégorielle de la cible
data_frame["stroke"] = data_frame["stroke"].astype("category")

## Compréhension des différentes variables

Analysons les variables à l'aide de graphiques.
Nous devons d'abord transformer les types ('object' et 'int') des variables catégorielles en type 'category' pour faciliter les analyses.

In [ ]:
# Transformation en type category de tous les variables catégorielles
for column in categorical_columns:
    data_frame[column] = data_frame[column].astype('category')

In [ ]:
data_frame[categorical_columns].dtypes

### Variables catégorielles

Identifions la proportion de chaque classe.

In [ ]:
# Récupérons les proportions des variables dans une liste
proportions = []
for column in categorical_columns:
    proportion = data_frame[column].value_counts(normalize = True)*100
    proportions.append(proportion)
    print(f"Colonne {column} :\n{proportion}\n-------------------\n")

In [ ]:
# Tracons le diagramme en barres des proportions pour chaque variable
fig, axs = plt.subplots(4, 2, figsize = (16, 14))
axs = axs.flat

for i, p in enumerate(proportions):
    fig.tight_layout(pad = 3)
    sns.barplot(x = p.index, y = p.values, ax = axs[i])
    axs[i].set_title(f"Diagramme en barres {p.name}")
    axs[i].set_xlabel(p.name)
    axs[i].set_ylabel("Pourcentages de valeurs")
    
fig.delaxes(axs[7])
    

### Variables non catégorielles

Comptons le nombre de patients par age à l'aide d'un countplot.

In [ ]:
plt.figure(figsize = (17, 17))
sns.countplot(data = data_frame, y = "age", palette = "viridis")
plt.title("Nombre de patients par age")

Nous constatons qu'il y a plus de patients adultes que de patients jeunes. Le plus grand nombre de tests a été effectué sur les personnes agées de 78 ans. Visualisons la distribution de la variable age. 

In [ ]:
plt.figure(figsize = (15, 8))
sns.histplot(data = data_frame, x = "age", kde = True)
plt.title("Distribution de la variable age")

La variable age ne suit pas une distribution normale. Nous remarquons que la distribution a tendance à être plus élevée au niveau des personnes agées entre 37 et 63 ans. La distribution est plus basse au niveau des jeunes patients.

Tracons les histogrammes et les densitées du niveau moyen de glucoses dans le sang et de l'indice de masse corporelle.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (17, 6))

axs = axs.flat

for i, column in enumerate(non_categorical_columns[-2:]):
    fig.tight_layout(w_pad = 3, pad = 1.2)
    color = "green" if i == 0 else "blue" 
    sns.histplot(data = data_frame, x = column, kde = True, ax = axs[i], color = color)
    

Nous remarquons, pour la variable avg_glucose_level, une composition de deux distributions (la deuxième à l'aire de se détacher de la première). La première distribution ressemble à une distribution normale avec une plus grande variance que la deuxième et sa moyenne tourne autour de 75-87 mg/dL de sang. Pour la deuxième distribution, dont la variance est plus faible, nous notons une moyenne tournant autour de 210-225 mg/dL.  

Nous remarquons, pour la variable bmi, une distribution qui semble être normale avec une moyenne tournant autour de 30 kg/m2.

## Utilisation de Pandas Profiling

Utilisons la librairie pandas profiling pour affiner nos analyses

In [ ]:
profile = ProfileReport(data_frame, title = "Investigation avec Pandas Profiling")

In [ ]:
profile

Nous reviendrons à quelques remarques effectuées par pandas profiling plus tard. Nous pouvons passer à l'analyse des relations entre les variables.

## Relations entre la variable cible et les variables explicatives

### Variables catégorielles / variable cible

Analysons ces relations en utilisant les tableaux croisés (de comptage) entre les variables.

In [ ]:
# Affichons des tableaux croisés entre la cible et chaque variable catégorielle à l'aide d'un heatmap.

fig, axs = plt.subplots(4, 2, figsize = (20, 16), sharey = True)
axs = axs.flat

for i, column in enumerate(categorical_columns):
    fig.tight_layout(pad = 3, h_pad = 4)
    sns.heatmap(pd.crosstab(data_frame["stroke"], data_frame[column]), annot = True, fmt = "d", ax = axs[i])
    axs[i].set_title(f"Tableau croisé entre stroke et {column}", fontsize = 17)
    axs[i].set_xlabel(column, fontsize = 14)
    
fig.delaxes(axs[7])

Nous remarquons que pour la variable work_type, on a une proportion de patients malades moins importante au niveau de la catégorie children.
Nous remarquons également que pour la variable smoking_status, la proportion de patients malades est moins importante au niveau de la catégorie unknown. Pour le reste des variables on ne peut rien dire pour l'instant.

### Variables non catégorielles / variable cible

Dans cette partie, nous devons affiner les analyses effectuées sur les variables non catégorielles en tracant leurs distributions suivant les différentes classes possibles de la variable cible. Nous verifierons ensuite si les distributions obtenues varient de la même manière. 

**Pour la variable age**

In [ ]:
plt.figure(figsize = (15, 6))
sns.histplot(data = data_frame, x = "age", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable age", fontsize = 14)
plt.xlabel("age", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous voyons que les deux distributions sont un peu différentes. Nous constatons que les personnes plus agées ont plus de risque d'être atteintes d'AVC.

**Pour la variable avg_glucose_level**

In [ ]:
plt.figure(figsize = (15, 8))
sns.histplot(data = data_frame, x = "avg_glucose_level", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable avg_glucose_level", fontsize = 14)
plt.xlabel("avg_glucose_level", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous ne remarquons aucune différence entre les deux distributions.

**Pour la variable bmi**. Remplacons les valeurs manquantes par le mode pour voir les distributions qu'on obtient.

In [ ]:
mode = data_frame["bmi"].mode()
data_to_plot = data_frame.fillna(mode)
plt.figure(figsize = (15, 8))
sns.histplot(data = data_to_plot, x = "bmi", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable bmi", fontsize = 14)
plt.xlabel("bmi", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous obtenons les mêmes distributions à première vue (il y a une nette différence mais pas très visible entre les deux distributions).

## Relations entre variables non catégorielles

Vérifions si certaines variables non catégorielles ont une forte corrélation entre elles.

In [ ]:
px.imshow(data_frame[non_categorical_columns].corr().round(2), text_auto = True)

Nous ne remarquons qu'aucune des coefficients de corrélation obtenues ne dépasse 0.5%. Les variables quantitatives ne partagent aucune corrélation entre elles.

## Relations entre variables catégorielles et non catégorielles

Nous allons vérifier pour chaque variable quantitative si elle est influencée par une ou plusieurs variable(s) catégorielle(s).

In [ ]:
def rel_cat_quant(categorical_column):
    """Fonction pour vérifier les relations pouvant exister entre les variables qualitatives
    et quantitatives. Nous allons utiliser la base de données ne comportant pas de valeurs 
    manquantes.
    
    Args:
        categorical_column(str): Nom de la variable catégorielle
    
    Returns:
        None
    """
    
    fig, axs = plt.subplots(2, 2, figsize = (18, 13))

    axs = axs.flat

    for i,column in enumerate(non_categorical_columns):

        sns.histplot(data = data_to_plot, x = column, kde = True, hue = categorical_column, ax = axs[i], palette = "tab10")

        axs[i].set_title(f"Variable {column}", fontsize = 14)

    fig.delaxes(axs[3])

    fig.tight_layout(pad = 3)

**Pour la variable hypertension**

In [ ]:
rel_cat_quant("hypertension")

Nous remarquons une **légère dépendance** entre les variables **hypertension** et **age**. 

**Pour la variable heart_disease**

In [ ]:
rel_cat_quant("heart_disease")

Nous constatons également une **légère dépendance** entre les variables **heart_disease** et **age**.

**Pour la variable gender**

In [ ]:
rel_cat_quant("gender")

Nous remarquons une **légère corrélation** (distributions un peu différentes) entre la variable **age** et **gender**.

**Pour la variable ever_married**

In [ ]:
rel_cat_quant("ever_married")

La variable **ever_married** est **fortement corrélée** avec les variables **age** et **bmi**.

**Pour la variable work_type**

In [ ]:
rel_cat_quant("work_type")

La variable **work_type** également réalise une **forte corrélation** entre les variables **age** et **bmi**. *Nous remarquons que la catégorie children de work_type désigne les patients qui sont des enfants (donc la variable work_type est cohérente pour le reste de l'analyse)*.

**Pour la variable residence_type**

In [ ]:
rel_cat_quant("Residence_type")

La variable **Residence_type** est **légèrement corrélée** (on peut même dire qu'elles ne sont pas corrélées) avec la variable **age**.

**Pour la variable smoking_status**. Remplacons la catégorie *unknown* par *never smoked* pour avoir une meilleure représentation de la variable.

In [ ]:
data_to_plot["smoking_status"] = data_to_plot["smoking_status"].apply(lambda x: "never smoked" if x == "Unknown" else x)

In [ ]:
data_to_plot["smoking_status"].value_counts(normalize=True)*100

In [ ]:
rel_cat_quant("smoking_status")

La variable **smoking_status** est **fortement corrélée** avec la variable **age**.

## Identification des valeurs aberrantes

Tracons les boxplots des variables catégorielles en les séparant par classe de patients (atteints d'AVC ou pas).

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (18, 14))

axs = axs.flat

for i,column in enumerate(non_categorical_columns):

    sns.boxplot(data = data_to_plot, x = column, hue = "stroke", ax = axs[i], palette = "tab10")

    axs[i].set_title(f"Variable {column}", fontsize = 14)

fig.delaxes(axs[3])

fig.tight_layout(pad = 3)

In [ ]:
def AberVal(dataframe, colonne):
    """Fonction pour retourner les valeurs abérantes et leurs index.
    
    Args:
        un Dataframe
        Colonnes (type : list)
    
    Returns:
        [index, value] (type : list)
    """
        
    index = []
    value = []
    for i in colonne:
        Q1 = data_frame[i].quantile(0.25)
        Q3 = data_frame[i].quantile(0.75)
        EIQ = Q3 - Q1
        LimitInf = Q1 - EIQ*1.5
        LimitSup = Q3 + EIQ*1.5

        idx = list(dataframe[i].index[(dataframe[i] < LimitInf) | (dataframe[i] > LimitSup)])
        val = dataframe[i][idx]
        index = list(set(index + idx))
        value = list(set(value + val))
    return [index, value]

In [ ]:
def deleteAberVal(dataframe, colonne):
    """Fonction pour supprimer les valeurs abérantes.
    
    Args:
        un Dataframe
        Colonnes (type : list)
        AberVal : function
    
    Returns:
        DataFrame
    """
    dataframe = dataframe.drop(index=AberVal(dataframe, colonne)[0])
    return dataframe

### Rensegner les valeur manquantes

In [ ]:
non_categorical = non_categorical.apply(lambda x:x.fillna(x.value_counts().index[0]))